In [ ]:
!pip install gradio
!pip install sumy

     |████████████████████████████████| 1.1MB 8.7MB/s 
     |████████████████████████████████| 215kB 36.7MB/s 
     |████████████████████████████████| 962kB 30.5MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 3.2MB 36.8MB/s 
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-cp37-none-any.whl size=3372 sha256=fa9c4b42579d94031100c7997dc08305eced127d74b9c9acc04d66d4a765c1e2
  Stored in directory: /root/.cache/pip/wheels/9f/fc/a7/ab5712c3ace9a8f97276465cc2937316ab8063c1fea488ea77
  Created wheel for ffmpy: filename=ffmpy-0.3.0-cp37-none-any.whl size=4710 sha256=de88f023d23bd15529a87a9bfdab7711202f4885e309a28dafe0fb44640a7cdf
  Stored in directory: /root/.cache/pip/wheels/cc/ac/c4/bef572cb7e52bfca170046f567e64858632daf77e0f34e5a74
  Created wheel for Flask-BasicAuth: filename=Flask_BasicAuth-0.2.0-cp37-none-any.whl size=4232 sha256=35b7f9eda6e0a6041f98c2b98d503e00ec59c0f86a1e60e98331890f3ff87230
  Stored in directory

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


import heapq
import gradio as gr


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def summary(text,choice):
  if choice == "word_freq":
    article_text = re.sub(r'\s+', ' ', text) #replace multiple whitespace with single space
    # Removing special characters and digits
    formatted_article_text = re.sub('[^a-zA-Z,.]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    #Converting Text To Sentences

    sentence_list = nltk.sent_tokenize(formatted_article_text)
    
    word_freq = {}
    sent_scores = {}

    for word in word_tokenize(formatted_article_text):
      if word.lower() not in stopwords:
        if word not in word_freq.keys():
          word_freq[word] = 1
        else:
          word_freq[word] += 1
    max_freq = max(word_freq.values()) #max freq from the frequencies of words
    for word in word_freq.keys():
      word_freq[word] = (word_freq[word]/max_freq)
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_freq.keys():
              if sent not in sent_scores.keys():
                sent_scores[sent] = word_freq[word]
              else:
                sent_scores[sent] += word_freq[word]

    summary_sentences = heapq.nlargest(7, sent_scores, key=sent_scores.get)
    summary_final = ' '.join(summary_sentences)
    return summary_final

  elif choice =="LexRank":

    parser = PlaintextParser.from_string(text,Tokenizer('english'))
    summarizer = LexRankSummarizer()
    lexRank_summary = summarizer(parser.document, 7) 

    summary_final = ""
    for sentence in lexRank_summary:
      summary_final += " "+str(sentence)
    return summary_final



In [ ]:
iface = gr.Interface(fn=summary, 
   inputs = [gr.inputs.Textbox(lines=5, placeholder="Enter Text Here..."), gr.inputs.Radio(["word_freq", "LexRank"])],outputs="text")

iface.launch(debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://36573.gradio.app
Interface loading below...
